In [1]:
## IMPORTS
import random
from deap import base
from deap import creator
from deap import tools
import numpy as np

In [2]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

In [3]:
def generate_random_num_attr():
  original = np.array([0.8, 2,0.1, 1.03, 1.1, 1.05, 0.975, 1])
  noises = []
  sum_list = []

  noises.append(np.random.normal(0, 0.05))
  noises.append(np.random.normal(0, 0.1))
  noises.append(np.random.normal(0, 0.01))
  noises.append(np.random.normal(0, 0.001))
  noises.append(np.random.normal(0, 0.01))
  noises.append(np.random.normal(0, 0.005))
  noises.append(np.random.normal(0, 0.005))
  noises.append(np.random.normal(0, 0.01))

  for (item1, item2) in zip(original, noises):
      sum_list.append(item1+item2)
  
  return sum_list

In [4]:
toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_bool", generate_random_num_attr)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, 1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [5]:
## change to our functinon
def evalOneMax(individual):
    return sum(individual[0]),

In [6]:
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [7]:
def main():
    pop = toolbox.population(n=300)
    
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
        
    # Begin the evolution
    for g in range(40):
        print("-- Generation %i --" % g)
        
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.5:
                toolbox.mate(child1[0], child2[0])
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < 0.2:
                toolbox.mutate(mutant[0])
                del mutant.fitness.values        
    
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        # Replace population
        pop[:] = offspring
    
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)
        
    print("-- End of (successful) evolution --")
    
    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))

In [8]:
main()

-- Generation 0 --
  Min 5.003840508508879
  Max 8.343708702906408
  Avg 8.061585701819713
  Std 0.4356821841235154
-- Generation 1 --
  Min 5.349030578638317
  Max 8.34504416736402
  Avg 8.13230022372267
  Std 0.37488381372095375
-- Generation 2 --
  Min 6.016787789595545
  Max 8.429599761098157
  Avg 8.20171418833518
  Std 0.3033556538056527
-- Generation 3 --
  Min 5.271834806752327
  Max 8.447591290597368
  Avg 8.211330938512248
  Std 0.3886349371240867
-- Generation 4 --
  Min 4.358229566440613
  Max 8.478704336250528
  Avg 8.291868007875212
  Std 0.3194436946518804
-- Generation 5 --
  Min 6.158319905284824
  Max 8.478704336250528
  Avg 8.314960344271448
  Std 0.28729118347060295
-- Generation 6 --
  Min 4.176308461481243
  Max 8.49216983819623
  Avg 8.342896901987444
  Std 0.36048893087071265
-- Generation 7 --
  Min 5.155948561296865
  Max 8.49216983819623
  Avg 8.366388368013467
  Std 0.3597015319181084
-- Generation 8 --
  Min 5.13678884382007
  Max 8.492439801675493
  Avg 8.